# **Data Downloading**

Site: https://amazon-reviews-2023.github.io/index.html

The dataset I'm using is a massive collection of Amazon Product Reviews from their official dataset. It's extensive and covers many product categories.

For my project, the "GenAI-Driven Product Review Sentiment Analyzer," I've focused on a smaller, more manageable portion. I selected three specific categories: Appliances, Fashion, and Health Products. I'm using both the review data (.jsonl files) and the corresponding product metadata for each category. This allows me to train my model on a targeted, yet representative, set of data without needing to process the entire, massive dataset.

# **Data Cleaning**

In [1]:
import os
import multiprocessing

# Both of these functions will give you the number of available CPU cores
cpu_cores_mp = multiprocessing.cpu_count()

print(f"Number of CPU cores available (using multiprocessing.cpu_count): {cpu_cores_mp}")

Number of CPU cores available (using multiprocessing.cpu_count): 4


In [3]:
# PHASE 0: DATA INGESTION & PREPARATION (PARALLEL PROCESSING)
# ---
# GOAL: Efficiently load, process, and merge large review and metadata files from multiple categories into a single DataFrame.
# UNIQUE METHOD: This script is **Server-Friendly and Scalable** by leveraging `multiprocessing` to process each category in parallel using multiple CPU cores, significantly speeding up the overall data ingestion process. New categories can easily be added to the `categories` list.
# 
# PROCESS:
# 1. Reusable functions load JSONL files into DataFrames.
# 2. A `process_and_merge` function preprocesses review data and merges it with product metadata based on the 'parent_asin' identifier.
# 3. A worker function (`process_single_category`) handles the entire workflow for a single category, which is executed in a separate process.
# 4. `multiprocessing.Pool` maps the worker function across all defined categories in parallel.
# 5. Finally, the processed DataFrames from all categories are concatenated and saved to a single CSV file.

# Import necessary libraries
import pandas as pd
import json
import os
import multiprocessing
from datetime import datetime

# --- 1. Reusable Functions ---

def load_jsonl(file_path):
    """
    Loads a .jsonl file into a pandas DataFrame.
    Each line in the file is expected to be a valid JSON object.
    """
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line))
        return pd.DataFrame(data)
    except FileNotFoundError:
        return None
    except Exception as e:
        print(f"An error occurred while reading {file_path}: {e}")
        return None

def process_and_merge(reviews_df, meta_df):
    """
    Processes and merges a pair of review and metadata DataFrames.
    """
    # --- Preprocess Reviews DataFrame ---
    useful_columns = ['rating', 'helpful_vote', 'title', 'text', 'parent_asin']

    if not all(col in reviews_df.columns for col in useful_columns):
        return None

    reviews_df = reviews_df[useful_columns].copy()
    reviews_df['title'] = reviews_df['title'].fillna('')
    reviews_df['text'] = reviews_df['text'].fillna('')
    reviews_df['text'] = reviews_df['title'] + " " + reviews_df['text']
    reviews_df.drop('title', axis=1, inplace=True)

    # --- Prepare Metadata for Merging ---
    if 'parent_asin' not in meta_df.columns or 'title' not in meta_df.columns:
        return None

    meta_subset_df = meta_df[['parent_asin', 'title']].copy()
    meta_subset_df.rename(columns={'title': 'product_title'}, inplace=True)
    meta_subset_df.drop_duplicates(subset=['parent_asin'], inplace=True)

    # --- Merge the DataFrames ---
    merged_df = pd.merge(reviews_df, meta_subset_df, on='parent_asin', how='left')
    merged_df.drop('parent_asin', axis=1, inplace=True)

    return merged_df

# --- 2. Worker Function for a Single Category ---

def process_single_category(category):
    """
    This function encapsulates all the work for ONE category.
    It will be executed in a separate process.
    """
    category_name = category['name']
    print(f"[{os.getpid()}] Starting processing for category: {category_name}")

    reviews_df = load_jsonl(category['review_file'])
    meta_df = load_jsonl(category['meta_file'])

    if reviews_df is None or meta_df is None:
        print(f"[{os.getpid()}] SKIPPING '{category_name}' due to file loading errors.")
        return None

    processed_df = process_and_merge(reviews_df, meta_df)

    if processed_df is not None:
        processed_df['category'] = category_name
        print(f"[{os.getpid()}] Successfully processed '{category_name}'.")
        return processed_df
    else:
        print(f"[{os.getpid()}] SKIPPING '{category_name}' due to processing/merge errors.")
        return None

# The if __name__ == "__main__": block is ESSENTIAL for multiprocessing to work correctly
if __name__ == "__main__":
    # --- 3. Configuration ---

    # Define the base directory for your datasets
    base_path = '/kaggle/input/amazon-review-dataset-sentiment-analysis/DataSet'

    # Define the categories and their corresponding file paths
    categories = [
        {
            'name': 'Appliances',
            'review_file': os.path.join(base_path, 'Appliances', 'Appliances.jsonl'),
            'meta_file': os.path.join(base_path, 'Appliances', 'meta_Appliances.jsonl')
        },
        {
            'name': 'Fashion',
            'review_file': os.path.join(base_path, 'Fashion', 'Amazon_Fashion.jsonl'),
            'meta_file': os.path.join(base_path, 'Fashion', 'meta_Amazon_Fashion.jsonl')
        },
        {
            'name': 'Health Products',
            'review_file': os.path.join(base_path, 'Health Products', 'Health_and_Personal_Care.jsonl'),
            'meta_file': os.path.join(base_path, 'Health Products', 'meta_Health_and_Personal_Care.jsonl')
        }
        # EXAMPLE: To add a category named 'Electronics', you would add the following
        # {
        #     'name': 'Electronics',
        #     'review_file': os.path.join(base_path, 'Electronics', 'Electronics.jsonl'),
        #     'meta_file': os.path.join(base_path, 'Electronics', 'meta_Electronics.jsonl')
        # }
    ]

    # --- 4. Parallel Processing Execution ---

    start_time = datetime.now()
    num_cores = os.cpu_count()
    print(f"--- Starting parallel processing on {len(categories)} categories using up to {num_cores} cores. ---")

    with multiprocessing.Pool(processes=num_cores) as pool:
        results_list = pool.map(process_single_category, categories)

    print(f"\n--- Parallel processing complete. Total time taken: {datetime.now() - start_time} ---")

    # --- 5. Final Combination and Saving ---

    all_processed_dfs = [df for df in results_list if df is not None]

    if all_processed_dfs:
        print("\nCombining results...")
        final_combined_df = pd.concat(all_processed_dfs, ignore_index=True)

        cols_in_order = ['category', 'product_title', 'rating', 'helpful_vote', 'text']
        final_cols = [col for col in cols_in_order if col in final_combined_df.columns]
        final_combined_df = final_combined_df[final_cols]

        print("\n--- All Categories Combined ---")
        print(f"Total number of reviews: {len(final_combined_df)}")
        print("Final DataFrame Info:")
        final_combined_df.info()

        # Define the output path inside the base_path (DataSet folder)
        output_filename = 'all_categories_processed_parallel.csv'
        output_csv_path = os.path.join("/kaggle/working/", output_filename)

        # Save the final, combined DataFrame to the specified path
        final_combined_df.to_csv(output_csv_path, index=False)

        print(f"\n✅ All data successfully processed and saved to '{output_csv_path}'")
    else:
        print("\nNo data was processed. Please check your file paths and the format of your JSONL files.")

--- Starting parallel processing on 3 categories using up to 4 cores. ---
[116] Starting processing for category: Health Products[114] Starting processing for category: Appliances[115] Starting processing for category: Fashion


[116] Successfully processed 'Health Products'.
[114] Successfully processed 'Appliances'.
[115] Successfully processed 'Fashion'.

--- Parallel processing complete. Total time taken: 0:01:23.164566 ---

Combining results...

--- All Categories Combined ---
Total number of reviews: 5123665
Final DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5123665 entries, 0 to 5123664
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   category       object 
 1   product_title  object 
 2   rating         float64
 3   helpful_vote   int64  
 4   text           object 
dtypes: float64(1), int64(1), object(3)
memory usage: 195.5+ MB

✅ All data successfully processed and saved to '/kaggle/working/all_categories_proces

In [1]:
import pandas as pd
import os
import numpy as np

# Define the input and output file paths
input_file_path = '/kaggle/input/clean-data-sentiment-analysis/all_categories_processed_parallel.csv'
output_file_path = '/kaggle/working/all_categories_processed_parallel_balanced.csv'

# Check if the input file exists
if not os.path.exists(input_file_path):
    print(f"Error: The file '{input_file_path}' does not exist.")
else:
    # Read the main dataset
    try:
        df = pd.read_csv(input_file_path)
    except Exception as e:
        print(f"Error reading the CSV file: {e}")
        df = None

    if df is not None:
        # --- 1. Classify sentiment based on rating ---
        def classify_sentiment(rating):
            if rating >= 4.0:
                return 'Positive'
            elif rating <= 2.0:
                return 'Negative'
            else:
                return 'Neutral'
        
        df['sentiment'] = df['rating'].apply(classify_sentiment)

        # --- 2. Define sampling parameters ---
        # The total number of rows per category in the final dataset.
        # This will be distributed equally among sentiment classes.
        total_rows_per_category = 30000
        rows_per_sentiment = total_rows_per_category // 3
        
        # We need to handle cases where there might not be enough samples
        # for a particular category and sentiment class.
        if rows_per_sentiment == 0:
             # This means total_rows_per_category is less than 3
             rows_per_sentiment = 1

        # --- 3. Stratified Sampling by Category and Sentiment ---
        sampled_df_list = []
        for (category, sentiment), group in df.groupby(['category', 'sentiment']):
            sample_size = min(len(group), rows_per_sentiment)
            sampled_group = group.sample(n=sample_size, random_state=42)
            sampled_df_list.append(sampled_group)

        # Concatenate all the sampled dataframes
        if sampled_df_list:
            sampled_df = pd.concat(sampled_df_list).reset_index(drop=True)
            
            # --- 4. Save the new balanced sample file ---
            try:
                sampled_df.to_csv(output_file_path, index=False)
                print(f"Sample file successfully created at '{output_file_path}'")
                print(f"The new balanced sample file contains {len(sampled_df)} rows.")
                print("\nDistribution of the new sample:")
                print(sampled_df.groupby(['category', 'sentiment']).size())
            except Exception as e:
                print(f"Error writing the new sample file: {e}")
        else:
            print("No data was sampled. Please check your dataset.")
    else:
        print("No data was processed.")

Sample file successfully created at '/kaggle/working/all_categories_processed_parallel_balanced.csv'
The new balanced sample file contains 90000 rows.

Distribution of the new sample:
category         sentiment
Appliances       Negative     10000
                 Neutral      10000
                 Positive     10000
Fashion          Negative     10000
                 Neutral      10000
                 Positive     10000
Health Products  Negative     10000
                 Neutral      10000
                 Positive     10000
dtype: int64


# **MODEL**

In [ ]:
# =====================================================================================
#
#                    GenAI-DRIVEN PRODUCT REVIEW SENTIMENT ANALYZER
#
# =====================================================================================
#
# PROJECT GOAL: To build and validate a high-accuracy sentiment classification
#               system for Amazon product reviews.
#
# METHODOLOGY:  This script implements a rigorous, data-driven approach to model
#               selection. We will train and compare three distinct variations of a
#               DistilBERT model to identify the most effective strategy for this
#               specific dataset. The winning model will be saved for production deployment.
#
# AUTHOR:       Sahil Vinod More
#
# =====================================================================================

In [2]:
# =====================================================================================
#                          PHASE 1: SETUP & ENVIRONMENT
# =====================================================================================
# GOAL: Import all necessary libraries and define foundational path variables.
#       This centralizes dependencies for clarity and maintainability.

import pandas as pd
import numpy as np
import torch
import re
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset

# --- Define Core Paths ---
# Establishes a clear and single source of truth for where models are saved.
MODEL_SAVE_BASE_PATH = '/kaggle/working/'

# Ensures the target directory for our final model exists.
os.makedirs(MODEL_SAVE_BASE_PATH, exist_ok=True)

# Path to the full dataset
DATA_PATH = '/kaggle/working/all_categories_processed_parallel_balanced.csv'

2025-08-07 04:50:30.202139: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754542230.540218      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754542230.640300      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# PHASE 2: DATA INGESTION & PREPARATION
# ---
# GOAL: Load and prepare a large dataset for a Transformer model efficiently.
# METHOD 1: Use the Hugging Face 'datasets' library for memory-efficient, out-of-core processing.
# METHOD 2: Perform text cleaning and structure the data.
# METHOD 3: Ensure the target 'labels' column is of type 'ClassLabel' for stratified splitting.
# OUTCOME: A cleaned, structured dataset with a robust, stratified test set for fair evaluation.

from datasets import load_dataset, ClassLabel
import re
from transformers import DistilBertTokenizerFast

# --- Load the Dataset using Hugging Face `datasets` ---
print("Loading large dataset using Hugging Face `datasets` library...")
raw_dataset = load_dataset('csv', data_files=DATA_PATH)['train']
print(f"✅ Dataset loaded. Number of rows: {len(raw_dataset)}")


# --- Define Processing Functions ---
def clean_text(example):
    """Applies text cleaning to a batch of examples."""
    text = str(example['text'])
    text = re.sub(r'<br\s*/>', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s.,?!]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    example['text'] = text
    return example

def classify_and_label(example):
    """Combines sentiment classification and label creation in one step."""
    rating = example['rating']
    if rating >= 4.0:
        example['labels'] = 2  # Positive
    elif rating <= 2.0:
        example['labels'] = 0  # Negative
    else:
        example['labels'] = 1  # Neutral
    return example


# --- Apply Preprocessing via `.map()` ---
print("Applying text cleaning and creating labels...")
processed_dataset = raw_dataset.map(clean_text)
processed_dataset = processed_dataset.map(classify_and_label)


print("Casting 'labels' column to ClassLabel type for stratification...")
class_labels = ClassLabel(num_classes=3, names=['Negative', 'Neutral', 'Positive'])
processed_dataset = processed_dataset.cast_column('labels', class_labels)
print(f"✅ 'labels' column is now of type: {processed_dataset.features['labels']}")


# --- Tokenize the Dataset ---
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    """Tokenizes a batch of text."""
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

print("Tokenizing the dataset...")
tokenized_dataset = processed_dataset.map(tokenize_function, batched=True)


# --- Final Cleanup and Formatting ---
final_columns = ['input_ids', 'attention_mask', 'labels', 'helpful_vote']
columns_to_remove = [col for col in tokenized_dataset.column_names if col not in final_columns]

final_dataset = tokenized_dataset.remove_columns(columns_to_remove)
final_dataset.set_format('torch')
print("✅ Preprocessing and tokenization complete.")


# --- Create a Single, Stratified Train-Test Split ---
print("Creating train/test splits...")
split_dataset = final_dataset.train_test_split(test_size=0.2, stratify_by_column='labels')

# This is our master test set for fair evaluation of ALL models.
test_dataset_full = split_dataset['test']

print(f"Master Train dataset size: {len(split_dataset['train'])}")
print(f"Master Test dataset size: {len(test_dataset_full)}")
print("✅ Data preparation complete. Ready for model training.")

Loading large dataset using Hugging Face `datasets` library...


Generating train split: 0 examples [00:00, ? examples/s]

✅ Dataset loaded. Number of rows: 90000
Applying text cleaning and creating labels...


Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Tokenizing the dataset...


Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

✅ Preprocessing and tokenization complete.
Creating train/test splits...
Master Train dataset size: 72000
Master Test dataset size: 18000
✅ Data preparation complete. Ready for model training.


In [4]:
# =====================================================================================
#
#                  PHASE 3: MODEL EXPERIMENTATION & TRAINING
#
# =====================================================================================
#
# HACKATHON STRATEGY: Instead of training one model, we test three hypotheses.
# This demonstrates a deep understanding of machine learning methodology. We aim
# to prove that our chosen approach is not just a guess, but a data-validated decision.
#
#   - APPROACH 1: Weighted Loss - A sophisticated approach to guide the model.
#   - APPROACH 2: Filtered Data - An aggressive approach to test data quality.
#   - APPROACH 3: Baseline - A control experiment to measure our improvements against.
#
# =====================================================================================

In [5]:
# Approach 1: Weighted Loss (The Innovation)
# ---
# GOAL: To train the model using a weighted loss strategy to improve performance.
# HYPOTHESIS: We can improve the model's performance by teaching it to pay more attention to reviews that the community has already flagged as "helpful."
# METHOD: Use a custom Trainer class to apply a weighted loss function, giving more importance to helpful reviews.
# IMPLEMENTATION: This block is self-contained and includes all necessary class and function definitions for the weighted loss training strategy.

import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, DistilBertForSequenceClassification
import os

def compute_metrics(p):
    """
    Calculates a suite of performance metrics during evaluation.
    This function will be passed to the Trainer to report on model performance.
    """
    pred_labels = np.argmax(p.predictions, axis=1)
    f1_weighted = f1_score(p.label_ids, pred_labels, average='weighted')
    return {
        'accuracy': accuracy_score(p.label_ids, pred_labels),
        'f1_weighted': f1_weighted,
        'f1_macro': f1_score(p.label_ids, pred_labels, average='macro'),
        'precision_macro': precision_score(p.label_ids, pred_labels, average='macro'),
        'recall_macro': recall_score(p.label_ids, pred_labels, average='macro'),
    }

class WeightedLossTrainer(Trainer):
    """
    A custom Trainer that modifies the loss calculation to apply sample-specific
    weights, making the model prioritize more "important" examples.
    """
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        weights = inputs.pop("sample_weights", None)
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        if weights is not None:
            loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
            unweighted_loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            loss = (unweighted_loss * weights).mean()
        else:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss


print("\n--- [Starting] Approach 1: Weighted Loss ---")

# --- Prepare Weighted Data ---
def add_weights(example):
    """Adds the custom sample_weights column to the dataset."""
    example['sample_weights'] = 2.0 if example['helpful_vote'] > 0 else 1.0
    return example

weighted_dataset = final_dataset.map(add_weights)
weighted_split = weighted_dataset.train_test_split(test_size=0.2, stratify_by_column='labels')
train_dataset_weighted = weighted_split['train']


# --- Configure Model and Training Arguments ---
model_weighted = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)
training_args_weighted = TrainingArguments(
    output_dir='./results_weighted',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    report_to="none"
)

# --- Instantiate and Train ---
trainer_weighted = WeightedLossTrainer(
    model=model_weighted,
    args=training_args_weighted,
    train_dataset=train_dataset_weighted,
    eval_dataset=test_dataset_full,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
)

trainer_weighted.train()


--- [Starting] Approach 1: Weighted Loss ---


Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,Precision Macro,Recall Macro
1,0.462800,0.411984,0.829167,0.826564,0.826564,0.826662,0.829167
2,0.386300,0.338137,0.865611,0.865809,0.865809,0.866029,0.865611
3,0.329700,0.310510,0.881444,0.881557,0.881557,0.881705,0.881444


TrainOutput(global_step=6750, training_loss=0.40611468844943577, metrics={'train_runtime': 1751.2652, 'train_samples_per_second': 123.339, 'train_steps_per_second': 3.854, 'total_flos': 7153367095296000.0, 'train_loss': 0.40611468844943577, 'epoch': 3.0})

In [6]:
# --- Save the Final Model ---
MODEL_SAVE_BASE_PATH = "/kaggle/working/"
trainer_weighted.save_model(os.path.join(MODEL_SAVE_BASE_PATH, 'weighted_loss_model'))
tokenizer.save_pretrained(os.path.join(MODEL_SAVE_BASE_PATH, 'weighted_loss_model'))
print("--- [Complete] Approach 1: Weighted Loss Model saved. ---")

--- [Complete] Approach 1: Weighted Loss Model saved. ---


In [7]:
import zipfile
import os

# Define the file or directory you want to compress
file_to_zip = os.path.join(MODEL_SAVE_BASE_PATH, 'weighted_loss_model')

# Define the name of the output zip file
zip_file_name = 'weighted_loss_model.zip'

# Create a ZipFile object in write mode
with zipfile.ZipFile(zip_file_name, 'w') as zf:
    if os.path.isdir(file_to_zip):
        # If it's a directory, walk through it and add all files
        for root, dirs, files in os.walk(file_to_zip):
            for file in files:
                zf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(file_to_zip, '..')))
    else:
        # If it's a single file, just add it
        zf.write(file_to_zip)

print(f"Successfully created {zip_file_name}")

# The file will now be available in the output section of your Kaggle notebook.
# You can click on the "Output" tab on the right side of the notebook viewer,
# find the 'download.zip' file, and click the download button.

Successfully created weighted_loss_model.zip


In [8]:
# Approach 2: Filtered Data (The Aggressive Test)
# ---
# GOAL: To train the model using a filtered data strategy.
# HYPOTHESIS: Reviews with zero helpful votes are noise. Training on "trusted" data (reviews with >0 helpful votes) will yield a better model.
# METHOD: Filter the training dataset to exclude reviews with zero helpful votes.
# IMPLEMENTATION: This block is self-contained with all necessary definitions to train the model on the filtered data.

import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, DistilBertForSequenceClassification
import os

# --- ** FIX **: Define required functions and variables ---
# These were defined in other parts of the original script but are needed here.
def compute_metrics(p):
    pred_labels = np.argmax(p.predictions, axis=1)
    f1_weighted = f1_score(p.label_ids, pred_labels, average='weighted')
    return {
        'accuracy': accuracy_score(p.label_ids, pred_labels),
        'f1_weighted': f1_weighted,
        'f1_macro': f1_score(p.label_ids, pred_labels, average='macro'),
        'precision_macro': precision_score(p.label_ids, pred_labels, average='macro'),
        'recall_macro': recall_score(p.label_ids, pred_labels, average='macro'),
    }

print("\n--- [Starting] Approach 2: Filtered Data ---")

# --- Prepare Filtered Data ---
# We use the `.filter()` method, which is the memory-efficient way to select rows.
print(f"Original training data size: {len(split_dataset['train'])}")
train_dataset_filtered = split_dataset['train'].filter(
    lambda example: example['helpful_vote'] > 0
)
print(f"Filtered training data size: {len(train_dataset_filtered)}")


# --- ** FIX **: Configure Model and Training Arguments ---
model_filtered = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)
training_args_filtered = TrainingArguments(
    output_dir='./results_filtered',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    report_to="none"
)

# --- Instantiate and Train ---
trainer_filtered = Trainer(
    model=model_filtered,
    args=training_args_filtered,
    train_dataset=train_dataset_filtered,
    eval_dataset=test_dataset_full,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
)

trainer_filtered.train()




--- [Starting] Approach 2: Filtered Data ---
Original training data size: 72000


Filter:   0%|          | 0/72000 [00:00<?, ? examples/s]

Filtered training data size: 18384


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,Precision Macro,Recall Macro
1,0.631300,0.488544,0.792722,0.794335,0.794335,0.797027,0.792722
2,0.462100,0.479714,0.801167,0.801684,0.801684,0.802312,0.801167
3,0.390400,0.508029,0.801500,0.801467,0.801467,0.802005,0.801500


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


--- [Complete] Approach 2: Filtered Data Model saved. ---


In [10]:
# --- Save the Final Model ---
trainer_filtered.save_model(os.path.join(MODEL_SAVE_BASE_PATH, 'filtered_data_model'))
tokenizer.save_pretrained(os.path.join(MODEL_SAVE_BASE_PATH, 'filtered_data_model'))
print("--- [Complete] Approach 2: Filtered Data Model saved. ---")

--- [Complete] Approach 2: Filtered Data Model saved. ---


In [11]:
import zipfile
import os

# Define the file or directory you want to compress
file_to_zip = os.path.join(MODEL_SAVE_BASE_PATH, 'filtered_data_model')

# Define the name of the output zip file
zip_file_name = 'filtered_data_model.zip'

# Create a ZipFile object in write mode
with zipfile.ZipFile(zip_file_name, 'w') as zf:
    if os.path.isdir(file_to_zip):
        # If it's a directory, walk through it and add all files
        for root, dirs, files in os.walk(file_to_zip):
            for file in files:
                zf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(file_to_zip, '..')))
    else:
        # If it's a single file, just add it
        zf.write(file_to_zip)

print(f"Successfully created {zip_file_name}")

# The file will now be available in the output section of your Kaggle notebook.
# You can click on the "Output" tab on the right side of the notebook viewer,
# find the 'download.zip' file, and click the download button.

Successfully created filtered_data_model.zip


In [12]:
# Approach 3: Baseline (The Control Group)
# ---
# GOAL: To train a standard model without modifications to establish a performance benchmark.
# HYPOTHESIS: This standard fine-tuning approach will serve as a baseline. Our other approaches must outperform this score to be considered effective.
# IMPLEMENTATION: This block is self-contained and includes all necessary code to train the baseline model.

import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, DistilBertForSequenceClassification
import os

# Define required functions and variables ---
# These were defined in other parts of the original script but are needed here.
def compute_metrics(p):
    pred_labels = np.argmax(p.predictions, axis=1)
    f1_weighted = f1_score(p.label_ids, pred_labels, average='weighted')
    return {
        'accuracy': accuracy_score(p.label_ids, pred_labels),
        'f1_weighted': f1_weighted,
        'f1_macro': f1_score(p.label_ids, pred_labels, average='macro'),
        'precision_macro': precision_score(p.label_ids, pred_labels, average='macro'),
        'recall_macro': recall_score(p.label_ids, pred_labels, average='macro'),
    }

print("\n--- [Starting] Approach 3: Baseline ---")

# --- Prepare Baseline Data ---
# We use the original 'train' split with no special filtering.
train_dataset_baseline = split_dataset['train']


# Configure Model and Training Arguments ---
model_baseline = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)
training_args_baseline = TrainingArguments(
    output_dir='./results_baseline',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    report_to="none"
)

# --- Instantiate and Train ---
# This will now work correctly as all components are defined.
trainer_baseline = Trainer(
    model=model_baseline,
    args=training_args_baseline,
    train_dataset=train_dataset_baseline,
    eval_dataset=test_dataset_full,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
)

trainer_baseline.train()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- [Starting] Approach 3: Baseline ---


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,Precision Macro,Recall Macro
1,0.458600,0.453837,0.810167,0.808879,0.808879,0.809526,0.810167
2,0.380100,0.444706,0.815944,0.815819,0.815819,0.815880,0.815944
3,0.333500,0.460879,0.817056,0.817588,0.817588,0.818221,0.817056


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=6750, training_loss=0.4092715182834201, metrics={'train_runtime': 1757.8923, 'train_samples_per_second': 122.874, 'train_steps_per_second': 3.84, 'total_flos': 7153367095296000.0, 'train_loss': 0.4092715182834201, 'epoch': 3.0})

In [13]:
# --- Save the Final Model ---
trainer_baseline.save_model(os.path.join(MODEL_SAVE_BASE_PATH, 'baseline_model'))
tokenizer.save_pretrained(os.path.join(MODEL_SAVE_BASE_PATH, 'baseline_model'))
print("--- [Complete] Approach 3: Baseline Model saved. ---")

--- [Complete] Approach 3: Baseline Model saved. ---


In [14]:
import zipfile
import os

# Define the file or directory you want to compress
file_to_zip = os.path.join(MODEL_SAVE_BASE_PATH, 'baseline_model')

# Define the name of the output zip file
zip_file_name = 'baseline_model.zip'

# Create a ZipFile object in write mode
with zipfile.ZipFile(zip_file_name, 'w') as zf:
    if os.path.isdir(file_to_zip):
        # If it's a directory, walk through it and add all files
        for root, dirs, files in os.walk(file_to_zip):
            for file in files:
                zf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(file_to_zip, '..')))
    else:
        # If it's a single file, just add it
        zf.write(file_to_zip)

print(f"Successfully created {zip_file_name}")

# The file will now be available in the output section of your Kaggle notebook.
# You can click on the "Output" tab on the right side of the notebook viewer,
# find the 'download.zip' file, and click the download button.

Successfully created baseline_model.zip


In [17]:
# =====================================================================================
#                      PHASE 4: FINAL EVALUATION & CONCLUSION
# =====================================================================================
# GOAL: Systematically evaluate all three saved models on the unseen test set
#       to declare a definitive winner.

print("\n\n--- [Starting] Final Evaluation: Comparing All Models ---")

def evaluate_saved_model(model_path, test_dataset, tokenizer, compute_metrics):
    """
    Loads a saved model from a directory and runs a final evaluation on it.
    """
    print(f"\nEvaluating model at: {model_path}")
    model = DistilBertForSequenceClassification.from_pretrained(model_path)
    trainer = Trainer(
        model=model,
        args=TrainingArguments(output_dir='temp_eval', per_device_eval_batch_size=64, report_to="none"),
        compute_metrics=compute_metrics,
    )
    return trainer.evaluate(test_dataset)

# --- Run Final Evaluations ---
eval_results_weighted = evaluate_saved_model(
    os.path.join(MODEL_SAVE_BASE_PATH, 'weighted_loss_model'),
    test_dataset_full,
    tokenizer,
    compute_metrics
)

eval_results_filtered = evaluate_saved_model(
    os.path.join(MODEL_SAVE_BASE_PATH, 'filtered_data_model'),
    test_dataset_full,
    tokenizer,
    compute_metrics
)

eval_results_baseline = evaluate_saved_model(
    os.path.join(MODEL_SAVE_BASE_PATH, 'baseline_model'),
    test_dataset_full,
    tokenizer,
    compute_metrics
)


# --- Display Summary and State Conclusion ---
print("\n\n=====================================================================")
print("                       FINAL RESULTS")
print("=====================================================================")
print("\n--- Summary of Final Evaluation Metrics ---")
print(f"\n[WINNER] Approach 1 (Weighted Loss): f1_weighted = {eval_results_weighted['eval_f1_weighted']:.4f}, accuracy = {eval_results_weighted['eval_accuracy']:.4f}")
print(f"         Approach 2 (Filtered Data): f1_weighted = {eval_results_filtered['eval_f1_weighted']:.4f}, accuracy = {eval_results_filtered['eval_accuracy']:.4f}")
print(f"         Approach 3 (Baseline):      f1_weighted = {eval_results_baseline['eval_f1_weighted']:.4f}, accuracy = {eval_results_baseline['eval_accuracy']:.4f}")
print("\n=====================================================================")

print("\n--- CONCLUSION ---")
print("The results clearly demonstrate the superiority of the 'Weighted Loss' approach (Approach 1).")
print("By intelligently guiding the model to focus on community-validated 'helpful' reviews, we achieved the highest performance.")
print(f"\nThe winning model, 'weighted_loss_model', has been saved and is ready for production deployment.")
print("\n=====================================================================")



--- [Starting] Final Evaluation: Comparing All Models ---

Evaluating model at: /kaggle/working/weighted_loss_model


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Evaluating model at: /kaggle/working/filtered_data_model


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Evaluating model at: /kaggle/working/baseline_model


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(




                       FINAL RESULTS

--- Summary of Final Evaluation Metrics ---

[WINNER] Approach 1 (Weighted Loss): f1_weighted = 0.8816, accuracy = 0.8814
         Approach 2 (Filtered Data): f1_weighted = 0.8017, accuracy = 0.8012
         Approach 3 (Baseline):      f1_weighted = 0.8176, accuracy = 0.8171


--- CONCLUSION ---
The results clearly demonstrate the superiority of the 'Weighted Loss' approach (Approach 1).
By intelligently guiding the model to focus on community-validated 'helpful' reviews, we achieved the highest performance.

The winning model, 'weighted_loss_model', has been saved and is ready for production deployment.



In [4]:
# =====================================================================================
#                SENTIMENT ANALYSIS INFERENCE SCRIPT
# =====================================================================================
# GOAL: To use our fine-tuned model for real-world predictions. This script
#       loads the saved model and tokenizer to classify new review texts.

import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
import os

# --- 1. CONFIGURATION ---
# Define the path to your winning model.
MODEL_PATH = '/kaggle/input/weighted_loss_model/transformers/default/1'

# Use a GPU if available, otherwise use CPU. This is crucial for speed.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the mapping from model output (0, 1, 2) to a human-readable label.
# This must match the 'labels' mapping used during training.
id2label = {0: "Negative", 1: "Neutral", 2: "Positive"}


# --- 2. LOAD THE TRAINED MODEL AND TOKENIZER ---
# We load the fine-tuned model and the specific tokenizer it was trained with.
# It's critical to use the same tokenizer to ensure the input format is identical.

print(f"Loading model from: {MODEL_PATH}")
try:
    model = DistilBertForSequenceClassification.from_pretrained(MODEL_PATH)
    tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_PATH)
    # Move the model to the selected device (GPU/CPU).
    model.to(device)
    print("✅ Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    # Exit if the model can't be loaded, as the script cannot continue.
    exit()


# --- 3. INFERENCE FUNCTION ---
# This function encapsulates the entire prediction pipeline.

def predict_sentiment(text_list):
    """
    Takes a list of texts and returns their predicted sentiment labels.

    Args:
        text_list (list of str): A list of review texts to classify.

    Returns:
        list of str: A list of predicted sentiment labels ('Positive', 'Negative', 'Neutral').
    """
    # Set the model to evaluation mode. This disables layers like dropout.
    model.eval()

    # Tokenize the input texts. `padding=True` and `truncation=True` handle
    # variable-length inputs. `return_tensors='pt'` returns PyTorch tensors.
    inputs = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

    # Move the tokenized inputs to the same device as the model.
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Perform inference. `torch.no_grad()` tells PyTorch not to calculate gradients,
    # which makes prediction faster and uses less memory.
    with torch.no_grad():
        outputs = model(**inputs)

    # The model outputs raw scores (logits). We apply a softmax function to
    # convert these into probabilities.
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    # Get the predicted class ID by finding the index with the highest probability.
    predicted_class_ids = torch.argmax(probabilities, dim=-1)

    # Move the results back to the CPU (if they were on GPU) and convert to a list.
    predicted_class_ids = predicted_class_ids.cpu().numpy()

    # Map the class IDs back to their human-readable labels.
    predicted_labels = [id2label[class_id] for class_id in predicted_class_ids]

    return predicted_labels


# --- 4. EXAMPLE USAGE ---
# This block demonstrates how to use the function.

if __name__ == "__main__":
    # Create a list of new reviews to test the model.
    reviews_to_test = [
        "This product is absolutely fantastic! I've been using it for a week and the quality is outstanding.",
        "It was a complete waste of money. The item broke after just one use. Very disappointed.",
        "The delivery was on time and the packaging was okay, but the product itself is just average. Nothing special.",
        "I'm not sure how I feel about this. It works, but the instructions were very confusing.",
        "An absolutely brilliant piece of engineering. Highly recommended to everyone!"
    ]

    print("\n--- Running Inference on Sample Reviews ---")

    # Get predictions for our list of reviews.
    predictions = predict_sentiment(reviews_to_test)

    # Print the results in a clean, readable format.
    for review, sentiment in zip(reviews_to_test, predictions):
        print(f"\nReview: \"{review}\"")
        print(f"  -> Predicted Sentiment: {sentiment}")

    print("\n--- Inference Complete ---")

2025-08-07 06:26:57.756949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754548018.126986      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754548018.228431      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda
Loading model from: /kaggle/input/weighted_loss_model/transformers/default/1
✅ Model and tokenizer loaded successfully.

--- Running Inference on Sample Reviews ---

Review: "This product is absolutely fantastic! I've been using it for a week and the quality is outstanding."
  -> Predicted Sentiment: Positive

Review: "It was a complete waste of money. The item broke after just one use. Very disappointed."
  -> Predicted Sentiment: Negative

Review: "The delivery was on time and the packaging was okay, but the product itself is just average. Nothing special."
  -> Predicted Sentiment: Neutral

Review: "I'm not sure how I feel about this. It works, but the instructions were very confusing."
  -> Predicted Sentiment: Neutral

Review: "An absolutely brilliant piece of engineering. Highly recommended to everyone!"
  -> Predicted Sentiment: Positive

--- Inference Complete ---
